---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd
pd.options.display.max_colwidth = 500

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0                        03/25/93 Total time of visit (in minutes):\n
1                                      6/18/85 Primary Care Doctor:\n
2            sshe plans to move as of 7/8/71 In-Home Services: None\n
3                               7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (Numeric Scale): 7\n
5                                   .Per 7/06/79 Movement D/O note:\n
6      4, 5/18/78 Patient's thoughts about current substance abuse:\n
7        10/24/89 CPT Code: 90801 - Psychiatric Diagnosis Interview\n
8                                        3/7/86 SOS-10 Total Score:\n
9                            (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
import re
from datetime import datetime as dt
#len(a1)+len(a2)+len(a3)+len(a4)+len(a5)+len(a6)
#Para verificar elementos repetidos
#para ampliar la vista de columnas en la consola 
pd.set_option('display.max_rows', 501)
"""
dic = {}
for x in a12356_index:
    dic[x] = dic.get(x,0)+1
        
for x,y in dic.items():
    if y>1: print(x,y)
"""

meses = {"Jan":1, "Feb":2, "Mar":3, "Apr":4, "May":5, "Jun":6, "Jul":7, "Aug":8, "Sep":9, "Oct":10, "Nov":11, "Dec":12}

def ciclo(actual,comp):
    a = []
    for x in actual:
        if x in comp.values: continue
        a.append(x)
    a = pd.Series(a)
    return a

def date_sorter():
    
    #04/20/2009; 04/20/09; 4/20/09; 4/3/09; 
    a1 = df.str.extractall(r"((\d{1,2})[/-](\d{1,2})[/-](\d{2}|\d{4})\b)")
    a1.reset_index(inplace=True); a1.set_index("level_0", inplace=True)
    a1_index = a1.reset_index().loc[:,"level_0"]
    
    #Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009; Mar-20-2009;
    a2 = df.str.extractall(r"((?:(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\w*[-.]* )(\d{1,2}[-,]* )(\d{2,4}))")
    a2.reset_index(inplace=True); a2.set_index("level_0", inplace=True)
    a2_index = a2.reset_index().loc[:,"level_0"]
    a12_index = a2_index.append(a1_index)
    
    #20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    #Feb 2009; Sep 2009; Oct 2010
    a3 = df.str.extractall(r"((\d{1,2} )?(?:(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\w*[-.,]* )(\d{2,4}))")
    a3.reset_index(inplace=True); a3.set_index("level_0", inplace=True)
    a3_index = a3.reset_index().loc[:,"level_0"]
    a3_index = ciclo(a3_index,a12_index)
    a3 = a3.loc[a3_index]
    a123_index = a12_index.append(a3_index) 
    
    #Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009 :::::::  NO existe
    a4 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z] )((?:\d{1,2}(?:st|nd|rd|th), )\d{4})')
    
    #6/2008; 12/2009
    a5 = df.str.extractall(r"((\d{1,2})/(\d{2,4}))")
    a5.reset_index(inplace=True); a5.set_index("level_0", inplace=True)
    a5_index = a5.reset_index().loc[:,"level_0"]
    a5_index = ciclo(a5_index,a123_index)
    a5 = a5.loc[a5_index]
    a1235_index = a123_index.append(a5_index)
    
    #2009; 2010
    a6 = df.str.extractall(r"((19[0-9][0-9])|(20[0-9][0-9]))")
    a6.reset_index(inplace=True); a6.set_index("level_0", inplace=True)
    a6_index = a6.reset_index().loc[:,"level_0"]
    a6_index = ciclo(a6_index,a1235_index)
    a6 = a6.loc[a6_index]
    
    #Manejo de los datos
    def primero(x):
        a = re.split("[/-]", x)
        if len(a[2])<4: a[2] = "19"+a[2]
        a = "/".join((a[0],a[1],a[2]))
        return a
            
    a1 = a1[0].apply(lambda x: primero(x))
    
    def segundo(x, y, z):
        a = x.split()
        a[0] = str(meses[y])
        if len(z)>2: z = z[0:2]
        a = "/".join((a[0],z,a[2]))
        return a
        
    a2 = a2.apply(lambda row: segundo(row[0],row[1],row[2]), axis=1)
    final = a1.append(a2)
    
    def tercero(x,y,z):
        y = str(meses[y])
        if len(x)>2: x = x[0:2]
        a = y+"/"+x+"/"+z
        return a
        
    a3 = a3.fillna("01")
    a3 = a3.apply(lambda row: tercero(row[1],row[2],row[3]), axis=1)
    final = final.append(a3)
    
    a5 = a5.apply(lambda row: row[1]+"/01/"+row[2], axis=1)
    final = final.append(a5)
    
    a6 = a6[0].apply(lambda x: "01/01/"+x)
    final = final.append(a6)
    
    final = final.apply(lambda x: dt.strptime(x, "%m/%d/%Y"))
    final = pd.to_datetime(final)
    final.sort_values(inplace=True)
    final = pd.Series(final.index)

    return final

In [3]:
dir(df)
pd.show_versions()


INSTALLED VERSIONS
------------------
commit: None
python: 3.6.0.final.0
python-bits: 64
OS: Linux
OS-release: 4.4.0-45-generic
machine: x86_64
processor: 
byteorder: little
LC_ALL: en_US.UTF-8
LANG: en_US.UTF-8
LOCALE: en_US.UTF-8

pandas: 0.20.1
pytest: 3.0.5
pip: 9.0.1
setuptools: 27.2.0
Cython: 0.25.2
numpy: 1.11.3
scipy: 0.18.1
xarray: None
IPython: 5.1.0
sphinx: 1.5.1
patsy: 0.4.1
dateutil: 2.6.0
pytz: 2016.10
blosc: None
bottleneck: 1.2.0
tables: 3.3.0
numexpr: 2.6.1
feather: None
matplotlib: 2.0.0
openpyxl: 2.4.1
xlrd: 1.0.0
xlwt: 1.2.0
xlsxwriter: 0.9.6
lxml: 3.7.2
bs4: 4.5.3
html5lib: None
sqlalchemy: 1.1.5
pymysql: None
psycopg2: None
jinja2: 2.9.4
s3fs: None
pandas_gbq: None
pandas_datareader: None
